In [1]:
# libs 
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords

nltk.download('all')
nltk.download('stopwords')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\sarah\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\sarah\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\sarah\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\sarah\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\sarah\AppData\Roaming\nltk_data...
[nltk_data]    | 

True

In [2]:
#read in data 
barbie_data = pd.read_csv("C:\\Users\\sarah\\OneDrive\\DS4002_Proj\\barbie_Cleaned.csv")
barbie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 797 entries, 0 to 796
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    797 non-null    object
 1   rating  797 non-null    object
dtypes: object(2)
memory usage: 12.6+ KB


In [3]:
barbie_data.head()

,text,rating
0,"Beautiful film, but so preachyLoveofLegacy21 J...",6
1,Clever and fun beginning turns into another le...,6
2,It was depressingcoxaneesa24 July 2023Warning:...,8
3,A Technicolor Dreamheatherhilgers24 July 2023W...,9
4,reasons FOR seeing it and 1 reason AGAINST.im...,7


In [4]:
#clean text

#separate conjoined words using wordninja 
import wordninja
barbie_data['text'] = barbie_data['text'].apply(lambda x: ' '.join(wordninja.split(x)))
barbie_data

,text,rating
0,Beautiful film but so preachy Love of Legacy 2...,6
1,Clever and fun beginning turns into another le...,6
2,It was depressing cox a nees a 24 July 2023 Wa...,8
3,A Technicolor Dream heather hil gers 24 July 2...,9
4,reasons FOR seeing it and 1 reason AGAINST im ...,7
...,...,...
792,Barbie not good thong man 22 July 2023 The onl...,2
793,Feminism but the rest iy as emin kos e 23 July...,5
794,Great sets great execution but horrible storyl...,3
795,Just meh april jose y 2903427 July 2023 Warnin...,2


In [5]:
#clean 

#remove non words 
import enchant 

def remove_non_english_words(text):
    english_dictionary = enchant.Dict("en_US")  # Use the English (US) dictionary

    separated_words = text.split()
    valid_words = [word for word in separated_words if english_dictionary.check(word)]

    return ' '.join(valid_words)


barbie_data['text'] = barbie_data['text'].apply(remove_non_english_words)
barbie_data

,text,rating
0,Beautiful film but so preachy Love of Legacy 2...,6
1,Clever and fun beginning turns into another le...,6
2,It was depressing cox a a 24 July 2023 Warning...,8
3,A Technicolor Dream heather 24 July 2023 Wow t...,9
4,reasons FOR seeing it and 1 reason AGAINST see...,7
...,...,...
792,Barbie not good thong man 22 July 2023 The onl...,2
793,Feminism but the rest as kos e 23 July 2023 I'...,5
794,Great sets great execution but horrible storyl...,3
795,Just meh y 2903427 July 2023 Warning Spoilers ...,2


In [6]:
#clean text 

#remove numbers
import re
barbie_data['text'] = barbie_data['text'].apply(lambda x: re.sub(r'\d+', '', x) if isinstance(x, str) else x)

barbie_data

,text,rating
0,Beautiful film but so preachy Love of Legacy ...,6
1,Clever and fun beginning turns into another le...,6
2,It was depressing cox a a July Warning Spoil...,8
3,A Technicolor Dream heather July Wow this mo...,9
4,reasons FOR seeing it and reason AGAINST see ...,7
...,...,...
792,Barbie not good thong man July The only reas...,2
793,Feminism but the rest as kos e July I'm NOT ...,5
794,Great sets great execution but horrible storyl...,3
795,Just meh y July Warning Spoilers out of fo...,2


In [12]:
#remove phrases
phrases_to_remove = ['found', 'helpful', 'review', 'Sign','vote', 'Permalink', 'Warning', 'Spoilers', 'July']
barbie_data['text'] = barbie_data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in phrases_to_remove]) if isinstance(x, str) else x)

barbie_data

,text,rating
0,Beautiful film but so preachy Love of Legacy M...,6
1,Clever and fun beginning turns into another le...,6
2,It was depressing cox a a out of this Was this...,8
3,A Technicolor Dream heather Wow this movie was...,9
4,reasons FOR seeing it and reason AGAINST see g...,7
...,...,...
792,Barbie not good thong man The only reason this...,2
793,Feminism but the rest as kos e Im NOT a Barbie...,5
794,Great sets great execution but horrible storyl...,3
795,Just meh y out of this Was this in to,2


In [8]:
#remove punctuation 
import string
barbie_data['text'] = barbie_data['text'].apply(lambda x: x.translate(str.maketrans("", "", string.punctuation)) if isinstance(x, str) else x)

barbie_data

,text,rating
0,Beautiful film but so preachy Love of Legacy M...,6
1,Clever and fun beginning turns into another le...,6
2,It was depressing cox a a out of found this he...,8
3,A Technicolor Dream heather Wow this movie was...,9
4,reasons FOR seeing it and reason AGAINST see g...,7
...,...,...
792,Barbie not good thong man The only reason this...,2
793,Feminism but the rest as kos e Im NOT a Barbie...,5
794,Great sets great execution but horrible storyl...,3
795,Just meh y out of found this helpful Was this ...,2


In [96]:
def remove_non_word_letters(text):
    words = wordninja.split(text)
    valid_words = [word for word in words if word.isalpha()]
    return ' '.join(valid_words)

# Apply the function to the 'text' column
barbie_data['text'] = barbie_data['text'].apply(remove_non_word_letters)
barbie_data

,text,rating
0,Beautiful film but so preachy Love Legacy Marg...,6
1,Clever and fun beginning turns into another le...,6
2,It was depressing cox a a in to,8
3,A Technicolor Dream heather Wow movie was a lo...,9
4,reasons FOR seeing it and reason AGAINST see g...,7
...,...,...
792,Barbie not good thong man The only reason movi...,2
793,Feminism but the rest as kos e Im NOT a Barbie...,5
794,Great sets great execution but horrible storyl...,3
795,Just meh y in to,2


In [73]:
#clean rating
barbie_data['rating'] = pd.to_numeric(barbie_data['rating'], errors = 'coerce')

#drop NaN
barbie_data = barbie_data.dropna(subset = ['rating'])

barbie_data['text']

0      Beautiful film but so preachy Love Legacy Marg...
1      Clever and fun beginning turns into another le...
2                        It was depressing cox a a in to
3      A Technicolor Dream heather Wow movie was a lo...
4      reasons FOR seeing it and reason AGAINST see g...
                             ...                        
792    Barbie not good thong man The only reason movi...
793    Feminism but the rest as kos e Im NOT a Barbie...
794    Great sets great execution but horrible storyl...
795                                     Just meh y in to
796    Not for little kids and wastes your time ben T...
Name: text, Length: 790, dtype: object

In [74]:
barbie_data['vader_scores'] = barbie_data['text'].apply(lambda x: SentimentIntensityAnalyzer().polarity_scores(x)['compound'])
barbie_data

C:\Users\sarah\AppData\Local\Temp\ipykernel_18060\248935630.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  barbie_data['vader_scores'] = barbie_data['text'].apply(lambda x: SentimentIntensityAnalyzer().polarity_scores(x)['compound'])


,text,rating,vader_scores
0,Beautiful film but so preachy Love Legacy Marg...,6.0,0.9857
1,Clever and fun beginning turns into another le...,6.0,0.8910
2,It was depressing cox a a in to,8.0,-0.3818
3,A Technicolor Dream heather Wow movie was a lo...,9.0,0.9635
4,reasons FOR seeing it and reason AGAINST see g...,7.0,0.0000
...,...,...,...
792,Barbie not good thong man The only reason movi...,2.0,0.5531
793,Feminism but the rest as kos e Im NOT a Barbie...,5.0,0.7879
794,Great sets great execution but horrible storyl...,3.0,0.9729
795,Just meh y in to,2.0,-0.0772


In [75]:
barbie_data['vader_scores'] = pd.to_numeric(barbie_data['vader_scores'])
barbie_data['rating'] = pd.to_numeric(barbie_data['rating'])

C:\Users\sarah\AppData\Local\Temp\ipykernel_18060\2472219045.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  barbie_data['vader_scores'] = pd.to_numeric(barbie_data['vader_scores'])
C:\Users\sarah\AppData\Local\Temp\ipykernel_18060\2472219045.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  barbie_data['rating'] = pd.to_numeric(barbie_data['rating'])


In [76]:
X = barbie_data['vader_scores']
y = barbie_data['rating']

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [78]:
X_train = X_train.values.reshape(-1, 1)
X_test = X_test.values.reshape(-1, 1)

In [79]:
#linear regression model 
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [80]:
predictions = model.predict(X_test)

In [82]:
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Mean Squared Error: {mse:.2f}')
print(f'R-squared: {r2:.2f}')

Mean Squared Error: 9.68
R-squared: 0.01


In [83]:
barbie_data.head(20)

,text,rating,vader_scores
0,Beautiful film but so preachy Love Legacy Marg...,6.0,0.9857
1,Clever and fun beginning turns into another le...,6.0,0.8910
2,It was depressing cox a a in to,8.0,-0.3818
3,A Technicolor Dream heather Wow movie was a lo...,9.0,0.9635
4,reasons FOR seeing it and reason AGAINST see g...,7.0,0.0000
5,Barbie Is A Weirdly Fun Movie Hakim While not ...,8.0,0.9987
6,Too heavy handed Nat cat in to,6.0,0.0000
7,Fun and surprisingly touching is my name I was...,8.0,0.9972
8,High Highs Low Lows The first thing you need t...,6.0,0.9947
9,Well really did come as a surprise Sleep in Dr...,8.0,0.9985
